In [0]:
%pip install transformers
%pip install sentence-transformers
%pip install huggingface_hub
%pip install accelerate
%pip install datasets
%pip install torch
dbutils.library.restartPython()


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql import functions as F

books_path   = "/Volumes/a/default/data/books/*.json"
reviews_path = "/Volumes/a/default/data/reviews/*.json"
authors_path = "/Volumes/a/default/data/goodreads_book_authors.json"
genres_path  = "/Volumes/a/default/data/goodreads_book_genres_initial.json"
series_path  = "/Volumes/a/default/data/goodreads_book_series.json"


# Load normally (no renaming of struct fields)
df_books  = spark.read.json(books_path)
df_reviews = spark.read.json(reviews_path)
df_authors = spark.read.json(authors_path)
df_genres  = spark.read.json(genres_path)
df_series = spark.read.json(series_path)


df_books = df_books.dropDuplicates().fillna("")
df_reviews = df_reviews.dropDuplicates().fillna("")
df_authors = df_authors.dropDuplicates().fillna("")
df_genres = df_genres.dropDuplicates().fillna("")
df_series = df_series.dropDuplicates().fillna("")

In [0]:
''' Test for genre array
from pyspark.sql.functions import *
exprs = [count(col(f"genres.`{c}`")).alias(c) for c in df_genres.select("genres.*").columns]
df_genres.select(*exprs).show(truncate=False)

from pyspark.sql import functions as F

df_test = df_genres.filter(F.col("book_id") == "102903").limit(1)

df_test = df_test.withColumn(
    "genre_array",
    F.array([
        F.when(
            F.col(f"genres.`{g}`").isNotNull() & (F.col(f"genres.`{g}`") > 0),
            F.lit(g)
        )
        for g in genre_fields
    ])
)
df_test = df_test.withColumn(
    "genre_array",
    F.expr("filter(genre_array, x -> x is not null)")
)
'''

' Test for genre array\nfrom pyspark.sql.functions import *\nexprs = [count(col(f"genres.`{c}`")).alias(c) for c in df_genres.select("genres.*").columns]\ndf_genres.select(*exprs).show(truncate=False)\n\nfrom pyspark.sql import functions as F\n\ndf_test = df_genres.filter(F.col("book_id") == "102903").limit(1)\n\ndf_test = df_test.withColumn(\n    "genre_array",\n    F.array([\n        F.when(\n            F.col(f"genres.`{g}`").isNotNull() & (F.col(f"genres.`{g}`") > 0),\n            F.lit(g)\n        )\n        for g in genre_fields\n    ])\n)\ndf_test = df_test.withColumn(\n    "genre_array",\n    F.expr("filter(genre_array, x -> x is not null)")\n)\n'

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import concat_ws, col, explode



#Explode authors from df_books + join df_authors metadata
df_books_author_rows = (
    df_books
    .select(
        "book_id",
        explode("authors").alias("author_struct")
    )
    .select(
        "book_id",
        col("author_struct.author_id").alias("author_id"),
        col("author_struct.role").alias("author_role"),
    )
)
df_author_flat = df_books_author_rows.join(df_authors, "author_id", "left")


df_reviews_flat = df_reviews.select(
    col("book_id"),
    col("review_text"),
    col("rating").alias("user_rating")
)

#df_series explode genre and only take value that is not NULL and then regroup again into string
genre_fields = df_genres.select("genres.*").columns

df_with_genre_array = df_genres.withColumn( # convert into array for ez filtering
    "genre_array",
    F.array([
        F.when(F.col(f"genres.`{g}`").isNotNull() & (F.col(f"genres.`{g}`") > 0), F.lit(g))
         for g in genre_fields
    ])
)

df_with_genre_array = df_with_genre_array.withColumn(
    "genre_array",
    F.expr("filter(genre_array, x -> x is not null)") #get rid of NULL values
)
df_series_flat = df_with_genre_array.withColumn(
    "genres_text",
    F.array_join("genre_array", ", ") #convert array into str
)

df_meta = (
    df_books
    .join(df_series_flat, "book_id", "left")       
    .join(df_reviews_flat, "book_id", "left")
    .join(df_author_flat, "book_id", "left")
)



In [0]:
# only select what important for a recommendation system embedding
df_meta = df_meta.withColumn(
    "embed_text",
    F.concat_ws(
        "\n",
        F.concat(F.lit("Title: "), col("title_without_series")),
        F.concat(F.lit("Author: "), col("name")),
        F.concat(F.lit("Genres: "), col("genres_text")),
        F.concat(F.lit("Description: "), col("description")),
        F.concat(F.lit("Review: "), col("review_text")),
        F.concat(F.lit("Series: "), col("title"))
    )
)

In [0]:
df_books = spark.read.json("/Volumes/a/default/data/books/goodreads_books_split_01.json")
df_books.printSchema()

root
 |-- asin: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- role: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- format: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- link: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- popular_shelves: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- count: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- pub

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, monotonically_increasing_id

CATALOG = "a"
SCHEMA = "default"
TABLE_NAME = f"{CATALOG}.{SCHEMA}.book_emb_test_50k"

MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
BATCH_SIZE = 64
REPARTITIONS = 32
LIMIT_ROWS = 50_000

print("Will save to:", TABLE_NAME)

df_50k = (
    df_meta
        .limit(LIMIT_ROWS)
        .withColumn("row_id", monotonically_increasing_id())
        .select(
            "book_id",
            "title_without_series",
            "genres_text",
            "description",
            "review_text",
            "name",
            "embed_text",
            "row_id"
        )
        .repartition(REPARTITIONS)
)

print("✔ Selected 50k rows.")


# ============================================================
# WORKER-LOCAL EMBEDDING
# ============================================================
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

def embed_pandas(batch_iter):
    model = SentenceTransformer(MODEL_NAME)

    for pdf in batch_iter:
        texts = pdf["embed_text"].fillna("").tolist()
        embeds = model.encode(texts, batch_size=BATCH_SIZE, show_progress_bar=False)
        pdf["embedding"] = np.asarray(embeds, dtype=np.float32).tolist()
        
        yield pdf


df_emb = df_50k.mapInPandas(
    embed_pandas,
    schema="""
        book_id STRING,
        title_without_series STRING,
        genres_text STRING,
        description STRING,
        review_text STRING,
        name STRING,
        embed_text STRING,
        row_id LONG,
        embedding ARRAY<FLOAT>
    """
)

print("✔ Embedding done.")


# ============================================================
# SAVE TO UNITY CATALOG
# ============================================================
df_emb.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(TABLE_NAME)

print("✔ Saved to:", TABLE_NAME)

Will save to: a.default.book_emb_test_50k
✔ Selected 50k rows.


2025-11-17 01:05:03.544869: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 01:05:03.549862: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 01:05:03.565021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-17 01:05:03.589674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-17 01:05:03.599559: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-17 01:05:03.623754: I tensorflow/core/platform/cpu_feature_gu

✔ Embedding done.
✔ Saved to: a.default.book_emb_test_50k


In [0]:
spark.sql("""
ALTER TABLE a.default.book_emb_test_50k
SET TBLPROPERTIES ('delta.enableChangeDataFeed' = 'true')
""")

DataFrame[]

In [0]:
from databricks.vector_search.client import VectorSearchClient
vs = VectorSearchClient()

index = vs.create_delta_sync_index(
    endpoint_name="books_endpoint",                           # your existing endpoint
    source_table_name="a.default.book_emb_test_50k",          # your table
    index_name="a.default.book_emb_test_50k_index",           # new index name
    primary_key="row_id",
    embedding_vector_column="embedding",
    embedding_dimension=384,                                  # MiniLM-L6 dimension
    pipeline_type="TRIGGERED"
)

index


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


In [0]:
index.describe()
index.sync()



{}

In [0]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

vec = model.encode("war").tolist()

results = index.similarity_search(
    query_vector=vec,
    columns=["book_id", "title_without_series", "genres_text"],
    num_results=5
)

results

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


{'manifest': {'column_count': 4,
  'columns': [{'name': 'book_id'},
   {'name': 'title_without_series'},
   {'name': 'genres_text'},
   {'name': 'score'}]},
 'result': {'row_count': 5,
  'data_array': [['1031322',
    'War Against War',
    'fiction, history, historical fiction, biography, non-fiction',
    0.4614063],
   ['18050121',
    'War Over The Word: How To Win the War You Are In',
    '',
    0.4612985],
   ['13593981',
    'The Making of the First World War',
    'history, historical fiction, biography, non-fiction',
    0.450767],
   ['1031322',
    'War Against War',
    'fiction, history, historical fiction, biography, non-fiction',
    0.44946077],
   ['101837',
    'The Warriors: Reflections on Men in Battle',
    'history, historical fiction, biography, non-fiction',
    0.44283143]]}}